In [ ]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-securi

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-01-18 02:21:23--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.36MB/s    in 0.2s    

2022-01-18 02:21:24 (5.36 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Software_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   42605767|R3EFW2STIYIY0I|B00MUTIDKI|     248732228|McAfee 2015 Inter...|        Software|          1|            2|          2|   N|                Y|I was very disapp...|I was very disapp...| 2015-08-31|
|         US|   51771800|R12NR0R5A9F7FT|B00EPACNUG|     531462352|Hallmark Card Stu...|        Software|          5|    

### Create DataFrames to match tables

In [ ]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame


In [ ]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3EFW2STIYIY0I|          1|            2|          2|   N|                Y|
|R12NR0R5A9F7FT|          5|            0|          0|   N|                Y|
|R1LSH74R9XAP59|          2|            0|          1|   N|                Y|
|R1QXUNTF76K7L6|          2|            0|          0|   N|                Y|
|R2F7DR75PS8NKT|          5|            0|          0|   N|                Y|
|R2C1DJSCC8UFS6|          3|            0|          0|   N|                Y|
|R1AXGS1W4YFXMX|          1|            0|          2|   N|                Y|
|R1XU1B93402SYJ|          1|            1|          1|   N|                Y|
|R2U432NB3OPVR0|          5|            0|          0|   N|                Y|
|R3R6FIMIOQ5SP9|          5|            0|          0|   N|     

### DELIVERABLE 2 

In [ ]:
# 1. get over 20 total votes
over_twenty_votes_df = vine_df.filter("total_votes >= 20")
over_twenty_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R26CC1STPSUHBS|          1|           27|         39|   N|                N|
| R7K83D59T9KWS|          1|           16|         26|   N|                N|
|R2ATG5BPVILHUM|          1|           16|         25|   N|                N|
|R1Q71TAM5ZOYTK|          3|           57|         61|   N|                N|
| RR8HYI7C2XKMU|          5|           25|         25|   N|                Y|
|R1VDCSJE4CJJIR|          1|           88|         97|   N|                Y|
|R2PZMSDKOO1Q6W|          1|           29|         43|   N|                Y|
|R39OOYN0DOLINK|          3|           91|        111|   N|                N|
|R1PLNHJZ8BKIBG|          1|           44|         46|   N|                Y|
|R14G4YQXL17REH|          1|            4|         28|   N|     

In [ ]:
to_pandas = over_twenty_votes_df.toPandas()
to_pandas.head()

,review_id,star_rating,helpful_votes,total_votes,vine,verified_purchase
0,R26CC1STPSUHBS,1,27,39,N,N
1,R7K83D59T9KWS,1,16,26,N,N
2,R2ATG5BPVILHUM,1,16,25,N,N
3,R1Q71TAM5ZOYTK,3,57,61,N,N
4,RR8HYI7C2XKMU,5,25,25,N,Y


In [ ]:
to_pandas['helpful_over_total'] = to_pandas["helpful_votes"]/to_pandas['total_votes']
to_pandas.head()

,review_id,star_rating,helpful_votes,total_votes,vine,verified_purchase,helpful_over_total
0,R26CC1STPSUHBS,1,27,39,N,N,0.692308
1,R7K83D59T9KWS,1,16,26,N,N,0.615385
2,R2ATG5BPVILHUM,1,16,25,N,N,0.640000
3,R1Q71TAM5ZOYTK,3,57,61,N,N,0.934426
4,RR8HYI7C2XKMU,5,25,25,N,Y,1.000000


In [ ]:
to_pandas_filtered = to_pandas[to_pandas['helpful_over_total'] >= 0.5]
to_pandas_filtered.head()

,review_id,star_rating,helpful_votes,total_votes,vine,verified_purchase,helpful_over_total
0,R26CC1STPSUHBS,1,27,39,N,N,0.692308
1,R7K83D59T9KWS,1,16,26,N,N,0.615385
2,R2ATG5BPVILHUM,1,16,25,N,N,0.640000
3,R1Q71TAM5ZOYTK,3,57,61,N,N,0.934426
4,RR8HYI7C2XKMU,5,25,25,N,Y,1.000000


In [ ]:
vine_y = to_pandas_filtered[to_pandas_filtered["vine"] == "Y"]
vine_y.head()

,review_id,star_rating,helpful_votes,total_votes,vine,verified_purchase,helpful_over_total
159,R28EIR3Z6RRUNN,2,50,60,Y,N,0.833333
205,R2YO2RESXZEWOF,4,24,25,Y,N,0.960000
247,R8GEE2EW3D6S9,3,49,54,Y,N,0.907407
255,R33W9V1MX6WKPS,4,47,53,Y,N,0.886792
274,R24E3W2ZRIFG8C,5,91,93,Y,N,0.978495


In [ ]:
vine_n = to_pandas_filtered[to_pandas_filtered["vine"] == "N"]
vine_n.head()

,review_id,star_rating,helpful_votes,total_votes,vine,verified_purchase,helpful_over_total
0,R26CC1STPSUHBS,1,27,39,N,N,0.692308
1,R7K83D59T9KWS,1,16,26,N,N,0.615385
2,R2ATG5BPVILHUM,1,16,25,N,N,0.640000
3,R1Q71TAM5ZOYTK,3,57,61,N,N,0.934426
4,RR8HYI7C2XKMU,5,25,25,N,Y,1.000000


In [ ]:
vine_y_review_count = vine_y["total_votes"].sum()
vine_y_review_count

20314

In [ ]:
vine_n_review_count = vine_n["total_votes"].sum()
vine_n_review_count

898388

In [ ]:
vine_y_fivestar_count = vine_y[vine_y["star_rating"] == 5].count().star_rating
vine_y_fivestar_count

102

In [ ]:
vine_n_fivestar_count = vine_n[vine_n["star_rating"] == 5].count().star_rating
vine_n_fivestar_count

5154

In [ ]:
vine_y_fivestar_count/vine_y_review_count

0.005021167667618391

In [ ]:
vine_n_fivestar_count / vine_n_review_count

0.005736942167526725